In [0]:
import time
import copy
from operator import add
from functools import reduce
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
sc=spark.sparkContext

In [0]:
toy_df = spark.sql("SELECT * FROM toytable_1_csv")

In [0]:
schema = toy_df.columns
schema

Out[97]: ['I1', 'I2', 'S1', 'S2', 'N1', 'N2']

In [0]:
LHS = {'I1'}
RHS = 'S2'

In [0]:
zeroVal1 = ([], 0)
mergeVal1 = (lambda aggregated, el: (aggregated[0] + [el[0]], aggregated[1] + el[1]))    
mergeComb1 = (lambda agg1,agg2:agg1+agg2)
zeroVal2 = ([], [], [])
mergeVal2 = (lambda aggregated, el: (aggregated[0] + [el[0]], aggregated[1] + [el[1]], aggregated[2] + [el[2]]))    
mergeComb2 = (lambda agg1,agg2:agg1+agg2)
zeroVal3 = ([], [])
mergeVal3 = (lambda aggregated, el: (aggregated[0] + [el[0]], aggregated[1] + [el[1]]))    
mergeComb3 = (lambda agg1,agg2:agg1+agg2)

In [0]:
# map and reduce into a five-tuple rdd
rdd1 = toy_df.rdd.map(lambda x: (*LHS, RHS, tuple(x[idx] for idx in list(map(lambda y: schema.index(y),LHS))), x[schema.index(RHS)]))\
    .map(lambda tpe: (tpe,1)).reduceByKey(add) \
    .map(lambda x: ((x[0][:-1]), (x[1], x[1])))\
    .aggregateByKey(zeroVal1,mergeVal1,mergeComb1)\
    .map(lambda x: ((x[0][:-1]), list(map(lambda r: r / x[1][1],x[1][0]))))\
    .map(lambda x: ((x[0]), [1 - reduce(lambda p1, p2:p1+p2,list(map(lambda p: p*(1-p), x[1])))]))\
    .reduceByKey(lambda x,y: x+y)\
    .filter(lambda tup: all(t >= 0.8 for t in tup[1]))\
#     .map(lambda tup: tup[0]).collect()
for line in rdd1.toLocalIterator():
    print(line)

In [0]:
def generate_computational_graph(RHS, schema):
    """
    Output
    ----------
    A dictionary where
    key: level
    value: list of current level's candidates, candidates are in the format of set
    -----

    """
    computational_graph=dict()
    for level in range(3):
        #use brute force to generate candidates for each level
        computational_graph[level]=[]
        if level== 0:
            for attribute  in schema:
                if attribute !=RHS:
                    computational_graph[level].append(set([attribute]))

        else:
            for element1 in computational_graph[level-1]:
                for element2 in computational_graph[0]:
                    newelement = element1.union(element2)
                    if newelement not in computational_graph[level]:
                        if len(newelement)==level+1:
                            computational_graph[level].append(newelement)    

    return computational_graph
def get_candidates(level, computational_graph):
    return computational_graph[level]
def prune_graph(level,current_level_result,computational_graph):
    """
    Input
    -------
    current_level_result: (soft/delta) functional dependencies discovered by algorithm, data structure: a list of candidates where candidates are in the format of sets
    computational_graph: A dict where key:level value: list of current level's candidates, candidates are in the format of set

    Output
    -------
    A pruned computational graph
    """
    # Candidates are pruned because minimal FD are already discovered

    # prune candidates after this level by verifying whether the next level has previous level's candidates as subset
    new_computational_graph = copy.deepcopy(computational_graph)
    while level<2:
        level+=1
        for LHS in current_level_result:
            for candidate in computational_graph[level]:
                if LHS.issubset(candidate):
                    if candidate in new_computational_graph[level]:
                        new_computational_graph[level].remove(candidate)


    return new_computational_graph
def transform_res(FDs):
    """
    Parameters
    --------------
    FDs: a list of (soft/delta) functional dependencies, where elements are tuples(LHS,RHS), LHS is in the format of set

    Output
    ---------
    current_level_result: a dictionary where key: RHS value: a list of LHS where candidates are in the form of sets
    """

    current_level_result=dict()
    for (LHS,RHS) in FDs:
        if RHS not in current_level_result.keys():
            current_level_result[RHS]=[]

        current_level_result[RHS].append(LHS)

    return current_level_result

In [0]:
def find_softFDs_pairs(level, df,current_level_candidates):
    softFDs=[]
    rdds=spark.sparkContext.emptyRDD()
    for RHS in current_level_candidates.keys():
        for LHS in current_level_candidates[RHS]:
            rddt=df.rdd.map(lambda x: (*LHS, RHS, tuple(x[idx] for idx in list(map(lambda y: schema.index(y),LHS))), x[schema.index(RHS)]))
            rdds=rdds.union(rddt)
    
    rdds = rdds.map(lambda tpe: (tpe,1)).reduceByKey(add)\
                .map(lambda x: ((x[0][:-1]), (x[1], x[1])))\
                .repartition(1)\
                .aggregateByKey(zeroVal1,mergeVal1,mergeComb1)\
                .map(lambda x: ((x[0][:-1]), list(map(lambda r: r / x[1][1],x[1][0]))))\
                .map(lambda x: ((x[0]), [1 - reduce(lambda p1, p2: p1+p2, list(map(lambda p: p*(1-p), x[1])))]))\
                .reduceByKey(lambda x,y: x+y)\
                .filter(lambda tup: all(t >= 0.8 for t in tup[1]))\
                .map(lambda x:(*x[0][:level],x[0][level])).collect()
                
    for item in rdds:
      softFDs.append(({*item[:-1]},item[-1]))

    return softFDs

In [0]:
def controller(df, func):
  """
  A control flow function

  Parameters
  -----------
  func: (soft/delta) Functional Discovery functions
  df: dataframe

  Output
  ------
  (soft/delta) Functional Dependencies
  """  
  # Initialization: Generate computational graph for each attribute which will be on RHS
  schema = df.columns
  computational_graph=dict()
  FDs=[]
  for RHS in schema:
    computational_graph[RHS]=generate_computational_graph(RHS,schema)

  for level in range(3):
    # Get current level candidates
    current_level_candidates=dict()
    for RHS in computational_graph.keys():
      current_level_candidates[RHS] = get_candidates(level,computational_graph[RHS])

  #     print('candidates:',current_level_candidates)
    # Use current_level candidates as an input to FD-functions for each level, func will return discovered (soft/delta)functional dependencies
    tFDs = func(level,df,current_level_candidates)
  #     print('FDs:',tFDs)
  #     print(tFDs)
    FDs.extend(tFDs)
    #Transform res into a dictionary where key: RHS value: a list of LHS where candidates are in the form of sets
    current_level_result = transform_res(tFDs)
  #     print(current_level_result)

    # Prune graphs according to feedback of FD-functions
  #     print(f"level:{level}, computatioanl_graph_key:{computational_graph.keys()},current_level_result_key:{current_level_result.keys()}")
    for RHS in computational_graph.keys():
      if RHS in current_level_result.keys():
        computational_graph[RHS]=prune_graph(level, current_level_result[RHS],computational_graph[RHS])


  return FDs

In [0]:
computational_graph=dict()
for RHS in schema:
    computational_graph[RHS]=generate_computational_graph(RHS,schema)

#Define current_level_candidates
current_level_candidates=dict()

for RHS in schema:
    current_level_candidates[RHS] = get_candidates(0,computational_graph[RHS])

In [0]:
rdds=spark.sparkContext.emptyRDD()
for RHS in current_level_candidates.keys():
    for LHS in current_level_candidates[RHS]:
        rddt=toy_df.rdd.map(lambda x: (*LHS, RHS, tuple(x[idx] for idx in list(map(lambda y: schema.index(y),LHS))), x[schema.index(RHS)]))
        rdds=rdds.union(rddt)

rdds = rdds.map(lambda tpe: (tpe,1)).reduceByKey(add)\
            .map(lambda x: ((x[0][:-1]), ((x[0][-1], x[1]), x[1])))\
            .repartition(4)\
            .aggregateByKey(zeroVal1,mergeVal1,mergeComb1)\
# .repartition(4)\
            
#             .map(lambda x: ((x[0]), list(map(lambda r: round(r[1] / x[1][1], 3),x[1][0]))))\
#             .map(lambda x: ((x[0]), 1 - reduce(lambda p1, p2:p1+p2,list(map(lambda p: p*(1-p), x[1])))))\
#             .map(lambda x: ((x[0][:-1]), (x[0][-1], x[1]))).aggregateByKey(zeroVal3,mergeVal3,mergeComb3)\
#             .filter(lambda tup: all(t >= 0.8 for t in tup[1][1]))\
#             .map(lambda x:(*x[0][:level],x[0][level]))

for item in rdds.toLocalIterator():
  print(item)

(('S1', 'I2', ('c',)), ([(1, 1), (2, 5)], 6))
(('N1', 'S1', (1.100000023841858,)), ([('a', 2)], 2))
(('I1', 'N1', (0,)), ([(1.100000023841858, 2)], 2, [(1.2000000476837158, 1)], 1))
(('I1', 'N2', (2,)), ([(3.0999999046325684, 2), (3.200000047683716, 4)], 6))
(('S2', 'I1', ('c',)), ([(2, 5)], 5))
(('N2', 'S1', (3.200000047683716,)), ([('c', 4)], 4))
(('N1', 'I2', (3.0999999046325684,)), ([(1, 1)], 1, [(2, 1)], 1))
(('S2', 'N1', ('c',)), ([(3.0999999046325684, 1)], 1, [(3.200000047683716, 4)], 4))
(('S1', 'N2', ('a',)), ([(1.100000023841858, 2)], 2, [(1.2000000476837158, 1)], 1))
(('N2', 'S2', (2.0999999046325684,)), ([('b', 1)], 1))
(('N2', 'I1', (2.0999999046325684,)), ([(1, 1)], 1))
(('S1', 'I2', ('b',)), ([(1, 2), (2, 1)], 3))
(('I2', 'I1', (1,)), ([(1, 2)], 2, [(2, 1)], 1))
(('N2', 'N1', (2.0999999046325684,)), ([(2.0999999046325684, 1)], 1))
(('N2', 'N1', (2.200000047683716,)), ([(2.200000047683716, 1)], 1))
(('I1', 'S1', (1,)), ([('b', 3)], 3))
(('N2', 'S2', (2.200000047683716,)), ([('b', 1)], 1))
(('I2', 'N1', (1,)), ([(2.0999999046325684, 1)], 1, [(3.0999999046325684, 1)], 1, [(2.200000047683716, 1)], 1))
(('N1', 'N2', (2.0999999046325684,)), ([(2.0999999046325684, 1)], 1))
(('N2', 'I1', (2.200000047683716,)), ([(1, 1)], 1))
(('N2', 'I1', (2.299999952316284,)), ([(1, 1)], 1))
(('N1', 'S1', (1.2000000476837158,)), ([('a', 1)], 1))
(('N2', 'I2', (1.100000023841858,)), ([(0, 2)], 2))
(('N2', 'I2', (1.2000000476837158,)), ([(0, 1)], 1))
(('N1', 'N2', (2.200000047683716,)), ([(2.200000047683716, 1), (2.299999952316284, 1)], 2))
(('I2', 'S2', (1,)), ([('a', 1)], 1, [('b', 2)], 2))
(('N1', 'I2', (3.200000047683716,)), ([(2, 4)], 4))
(('N2', 'S1', (3.0999999046325684,)), ([('c', 2)], 2))
(('S2', 'N1', ('b',)), ([(2.0999999046325684, 1), (2.200000047683716, 1)], 2))
(('N2', 'S2', (2.299999952316284,)), ([('a', 1)], 1))
(('N2', 'N1', (2.299999952316284,)), ([(2.200000047683716, 1)], 1))
(('I2', 'S1', (0,)), ([('a', 3)], 3))
(('I1', 'S2', (0,)), ([('a', 3)], 3))
(('S2', 'S1', ('a',)), ([('a', 3), ('c', 1)], 4, [('b', 1)], 1))
(('S2', 'I1', ('b',)), ([(1, 2)], 2))
(('N1', 'I1', (1.100000023841858,)), ([(0, 2)], 2))
(('N1', 'I2', (2.200000047683716,)), ([(2, 1)], 1, [(1, 1)], 1))
(('S1', 'N2', ('b',)), ([(2.299999952316284, 1), (2.0999999046325684, 1)], 2, [(2.200000047683716, 1)], 1))
(('N2', 'N1', (3.200000047683716,)), ([(3.200000047683716, 4)], 4))
(('S1', 'N2', ('c',)), ([(3.0999999046325684, 2)], 2, [(3.200000047683716, 4)], 4))
(('S2', 'I1', ('a',)), ([(0, 3), (2, 1), (1, 1)], 5))
(('N1', 'S2', (1.100000023841858,)), ([('a', 2)], 2))
(('N2', 'S2', (3.0999999046325684,)), ([('c', 1), ('a', 1)], 2))
(('I1', 'N1', (1,)), ([(2.200000047683716, 2)], 2, [(2.0999999046325684, 1)], 1))
(('I2', 'N1', (0,)), ([(1.2000000476837158, 1)], 1, [(1.100000023841858, 2)], 2))
(('N1', 'I1', (1.2000000476837158,)), ([(0, 1)], 1))
(('N2', 'I1', (3.200000047683716,)), ([(2, 4)], 4))
(('I2', 'N2', (2,)), ([(2.299999952316284, 1)], 1, [(3.200000047683716, 4)], 4, [(3.0999999046325684, 1)], 1))
(('N2', 'I1', (3.0999999046325684,)), ([(2, 2)], 2))
(('I2', 'S1', (1,)), ([('b', 2)], 2, [('c', 1)], 1))
(('I1', 'S1', (0,)), ([('a', 3)], 3))
(('N1', 'S2', (1.2000000476837158,)), ([('a', 1)], 1))
(('N2', 'N1', (3.0999999046325684,)), ([(3.0999999046325684, 2)], 2))
(('N1', 'N2', (3.0999999046325684,)), ([(3.0999999046325684, 2)], 2))
(('I1', 'I2', (2,)), ([(2, 5), (1, 1)], 6))
(('S2', 'N1', ('a',)), ([(2.200000047683716, 1), (1.2000000476837158, 1), (3.0999999046325684, 1)], 3, [(1.100000023841858, 2)], 2))
(('S1', 'I2', ('a',)), ([(0, 3)], 3))
(('I2', 'S2', (0,)), ([('a', 3)], 3))
(('N2', 'S1', (2.299999952316284,)), ([('b', 1)], 1))
(('S2', 'S1', ('c',)), ([('c', 5)], 5))
(('N1', 'N2', (3.200000047683716,)), ([(3.200000047683716, 4)], 4))
(('I2', 'I1', (0,)), ([(0, 3)], 3))
(('N2', 'S1', (2.0999999046325684,)), ([('b', 1)], 1))
(('I1', 'S2', (1,)), ([('b', 2), ('a', 1)], 3))
(('S2', 'S1', ('b',)), ([('b', 2)], 2))
(('N1', 'I2', (2.0999999046325684,)), ([(

In [0]:
start_time = time.time()
softFDs = find_softFDs_pairs(3, toy_df, current_level_candidates)
print("--- %s seconds ---" % (time.time() - start_time))
print(softFDs)

In [0]:
computational_graph=dict()
df_10000 = spark.sql("SELECT * FROM _2018_01_bme280sof_3_csv limit 2000")
df_10000=df_10000.drop('_c0')
schema = df_10000.columns
for RHS in schema:
    computational_graph[RHS]=generate_computational_graph(RHS,schema)

#Define current_level_candidates
current_level_candidates=dict()

for RHS in schema:
    current_level_candidates[RHS] = get_candidates(0, computational_graph[RHS])

In [0]:
start_time = time.time()
# softFDs = find_softFDs_pairs(1, df_10000, current_level_candidates)
softFDs = controller(df_10000, find_softFDs_pairs)
print("--- %s seconds ---" % (time.time() - start_time))
print(softFDs)

--- 92.60102319717407 seconds ---
[(set(), 'location'), (set(), 'sensor_id'), (set(), 'sensor_id'), (set(), 'location'), (set(), 'location'), (set(), 'sensor_id'), ({'lon'}, 'sensor_id'), ({'sensor_id'}, 'timestamp'), ({'sensor_id'}, 'pressure'), ({'sensor_id'}, 'temperature'), ({'sensor_id'}, 'humidity'), ({'lon'}, 'location'), ({'timestamp'}, 'location'), ({'location'}, 'pressure'), ({'location'}, 'temperature'), ({'humidity'}, 'location'), ({'timestamp'}, 'pressure'), ({'timestamp'}, 'temperature'), ({'humidity'}, 'pressure'), ({'sensor_id'}, 'location'), ({'sensor_id'}, 'timestamp'), ({'sensor_id'}, 'pressure'), ({'sensor_id'}, 'temperature'), ({'sensor_id'}, 'humidity'), ({'location'}, 'lat'), ({'timestamp'}, 'location'), ({'location'}, 'pressure'), ({'location'}, 'temperature'), ({'humidity'}, 'location'), ({'lat'}, 'pressure'), ({'timestamp'}, 'pressure'), ({'timestamp'}, 'temperature'), ({'humidity'}, 'pressure'), ({'sensor_id'}, 'timestamp'), ({'timestamp'}, 'location'), ({'timestamp'}, 'temperature'), ({'sensor_id'}, 'timestamp'), ({'timestamp'}, 'location'), ({'timestamp'}, 'pressure'), ({'humidity'}, 'pressure'), ({'sensor_id'}, 'timestamp'), ({'timestamp'}, 'location'), ({'timestamp'}, 'pressure'), ({'timestamp'}, 'temperature'), ({'sensor_id'}, 'location'), ({'sensor_id'}, 'lat'), ({'lon', 'timestamp'}, 'pressure'), ({'lon', 'timestamp'}, 'temperature'), ({'lon', 'timestamp'}, 'humidity'), ({'lon', 'temperature'}, 'pressure'), ({'lon', 'humidity'}, 'pressure'), ({'lon', 'humidity'}, 'temperature'), ({'timestamp', 'temperature'}, 'pressure'), ({'timestamp', 'humidity'}, 'pressure'), ({'timestamp', 'temperature'}, 'humidity'), ({'humidity', 'temperature'}, 'pressure'), ({'sensor_id', 'location'}, 'lat'), ({'sensor_id', 'timestamp'}, 'location'), ({'sensor_id', 'location'}, 'pressure'), ({'sensor_id', 'location'}, 'temperature'), ({'sensor_id', 'humidity'}, 'location'), ({'sensor_id', 'timestamp'}, 'lat'), ({'sensor_id', 'lat'}, 'pressure'), ({'sensor_id', 'lat'}, 'temperature'), ({'sensor_id', 'humidity'}, 'lat'), ({'sensor_id', 'timestamp'}, 'pressure'), ({'sensor_id', 'timestamp'}, 'temperature'), ({'sensor_id', 'timestamp'}, 'humidity'), ({'sensor_id', 'temperature'}, 'pressure'), ({'sensor_id', 'humidity'}, 'pressure'), ({'sensor_id', 'humidity'}, 'temperature'), ({'timestamp', 'location'}, 'lat'), ({'lat', 'location'}, 'pressure'), ({'lat', 'location'}, 'temperature'), ({'humidity', 'location'}, 'lat'), ({'timestamp', 'location'}, 'pressure'), ({'timestamp', 'location'}, 'temperature'), ({'timestamp', 'location'}, 'humidity'), ({'location', 'pressure'}, 'temperature'), ({'humidity', 'location'}, 'pressure'), ({'humidity', 'location'}, 'temperature'), ({'timestamp', 'lat'}, 'pressure'), ({'timestamp', 'lat'}, 'temperature'), ({'timestamp', 'lat'}, 'humidity'), ({'lat', 'pressure'}, 'temperature'), ({'humidity', 'lat'}, 'pressure'), ({'humidity', 'lat'}, 'temperature'), ({'timestamp', 'temperature'}, 'pressure'), ({'timestamp', 'humidity'}, 'pressure'), ({'timestamp', 'temperature'}, 'humidity'), ({'humidity', 'temperature'}, 'pressure'), ({'humidity', 'lat'}, 'pressure'), ({'lon', 'humidity'}, 'pressure'), ({'timestamp', 'lat'}, 'location'), ({'lon', 'timestamp'}, 'location'), ({'timestamp', 'location'}, 'pressure'), ({'timestamp', 'location'}, 'temperature'), ({'lon', 'timestamp'}, 'lat'), ({'timestamp', 'lat'}, 'pressure'), ({'timestamp', 'lat'}, 'temperature'), ({'lon', 'timestamp'}, 'pressure'), ({'lon', 'timestamp'}, 'temperature'), ({'timestamp', 'temperature'}, 'pressure')]